1) Load Data

In [1]:
import pandas as pd
import json

df = pd.read_csv("../data/raw/train.csv")
print(f"Before cleaning: {len(df)} rows")

Before cleaning: 1000 rows


2) Remove Duplicate Conversations

In [2]:
df = df.drop_duplicates(subset=['conversation'], keep='first')
print(f"After removing duplicate conversations: {len(df)} rows")

After removing duplicate conversations: 998 rows


3) Remove Very Short Conversations

In [3]:
df['conv_length'] = df['conversation'].str.len()
df = df[df['conv_length'] >= 100]
print(f"After removing short conversations: {len(df)} rows")
df = df.drop(columns=['conv_length'])

After removing short conversations: 997 rows


4) Remove Invalid QA Rows

In [4]:
def is_valid_qa(qa_str):
    try:
        qa = json.loads(qa_str)
        if 'knowledge' not in qa or len(qa['knowledge']) == 0:
            return False
        for pair in qa['knowledge']:
            q = pair.get('customer_summary_question', '').strip()
            a = pair.get('agent_summary_solution', '').strip()
            if not q or not a:
                return False
        return True
    except:
        return False

df = df[df['qa'].apply(is_valid_qa)]
print(f"After removing invalid QA rows: {len(df)} rows")

After removing invalid QA rows: 983 rows


5) Clean Text 

In [5]:
df['conversation'] = df['conversation'].str.strip()
df['conversation'] = df['conversation'].str.replace(r'\s+', ' ', regex=True)
print("✅ Cleaned extra whitespace from conversations")

✅ Cleaned extra whitespace from conversations


6) Extract QA Pairs into Separate Columns

In [6]:
def extract_qa(qa_str):
    qa = json.loads(qa_str)
    questions = []
    answers = []
    for pair in qa['knowledge']:
        questions.append(pair['customer_summary_question'].strip())
        answers.append(pair['agent_summary_solution'].strip())
    return questions, answers

df['questions'] = df['qa'].apply(lambda x: extract_qa(x)[0])
df['answers'] = df['qa'].apply(lambda x: extract_qa(x)[1])

print(f"Total QA pairs extracted: {df['questions'].apply(len).sum()}")
print(f"Avg pairs per row: {df['questions'].apply(len).mean():.1f}")

Total QA pairs extracted: 1881
Avg pairs per row: 1.9


7) Drop Redundant Columns

In [7]:
df_clean = df.drop(columns=['issue_category_sub_category', 'agent_experience_level_desc'])
print(f"Columns after dropping redundant ones: {df_clean.columns.tolist()}")

Columns after dropping redundant ones: ['issue_area', 'issue_category', 'issue_sub_category', 'customer_sentiment', 'product_category', 'product_sub_category', 'issue_complexity', 'agent_experience_level', 'conversation', 'qa', 'questions', 'answers']


8) Save

In [8]:
print("="*50)
print("CLEANING SUMMARY")
print("="*50)
print(f"Original rows: 1000")
print(f"Final rows: {len(df_clean)}")
print(f"Removed: {1000 - len(df_clean)} rows")
print(f"Columns: {len(df_clean.columns)}")
print(f"\nFinal columns: {df_clean.columns.tolist()}")

# Save cleaned data
df_clean.to_csv("../data/processed/cleaned_data.csv", index=False)
print("\n✅ Saved to data/processed/cleaned_data.csv")

CLEANING SUMMARY
Original rows: 1000
Final rows: 983
Removed: 17 rows
Columns: 12

Final columns: ['issue_area', 'issue_category', 'issue_sub_category', 'customer_sentiment', 'product_category', 'product_sub_category', 'issue_complexity', 'agent_experience_level', 'conversation', 'qa', 'questions', 'answers']

✅ Saved to data/processed/cleaned_data.csv


9) Check

In [9]:
check = pd.read_csv("../data/processed/cleaned_data.csv")
print(f"Saved file shape: {check.shape}")
print(f"Missing values: {check.isnull().sum().sum()}")
print("✅ All good!")

Saved file shape: (983, 12)
Missing values: 0
✅ All good!
